<a href="https://colab.research.google.com/github/MWSteams/4590FSP/blob/master/Chemicals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CHEMICALS**

## **Authors: Bridget Childs, Ananya Gangadhar, Jacqueline Wong**

# **1. Task Definition**

<p align="center"> 
<img src="https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/chemicals_task.png">
</p>

The Chemicals team will be in charge of designing the chemical dosage system. This will include: 

1. Determining coagulant and polymer dosage
2. Designing chemical storage, feed lines, and flow control system.
3. Calculating feed hydraulics and plumbing
4. Identifying available vendors.

Additionally, our storage system design will include the number of storage tanks, tank size, location, footprint, and safety add-ons in order to meet OSHA requirements (e.g. spillover capture, eye washes, safety showers). Information will be primarily sourced from government regulations, the Fairmont Sedimentation Plant Scope of Work document, relevant vendors, and similar go-by systems.


# **2. Executive Summary**

(Highlight dominant constraints and your recommendation for the design.)


![chemicals layout](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/chemicals_layout.png)

# **3. Constraints**

* Design flow of the plant is 720 cfs.
* Effluent arsenic levels should be under 10 ug/L.
* Effluent TOC levels should be under 3 mg/L.
* Effluent turbidity should be under 3 NTU.
* Chemicals will be stored in an open tank farm and have at least a 30 day capacity  (Scope of work, pg 18).
* Proposed tank farm should have area under 200'x100'
* All hazardous chemicals must be stored in an area that is designed to contain spills ([OSHA regulations](https://www.safetyinfo.com/safe-chemical-storage-osha-requirements-free-index/)).
* All design values should be justifiable using design equations.
* The chemical feed system should be easy to understand and operate, and should have relatively low maintenance.
* The designed system should have a feedback loop wherein the chemical dosage is adjusted accordingly with a change in the plant flow rate.

**CEE 4590 Logistical constraints**
* Information in Scope of Work is often missing citations, and we are unsure of the source and degree of certainty of parameters like expected FeCl3 dosage.
* At present, we have no communication with the LA engineers on this project, only the Stantec engineers, Mickey and Tori, who are working on Stantec's own proposal for this project. 
* It will be challenging to procure cost estimates from vendors since they are typically reluctant to share such information with students.


## **3.1. Table of potential strategies to meet the constraints and treatment objectives**

| strategy | advantages  | disadvantages | data needed |
|---|---|---|---|
| open tank farm vs. enclosed storage | go-by, less expensive | less protection | weathering risk |
| 3 storage tanks vs. using even more | less complexity | lower safety factor | storage area |
| choosing vendor before choosing tank size | easier to pick model | more restricted | vendor details |
| 30-day storage vs. keeping even more | resilience (e.g. COVID-19) | more expensive | area, budget |
| single vs. multiple dosage system per train | maintenance, operator ease | fewer options if pipe fails | clogging risk |

# **4. Physics**

Since the Chemicals Team task concerns mainly the storage and delivery of coagulant and polymers, it is important to consider the energy and space that these chemicals will occupy. The **Energy equation** accounts the energy losses in fluid mechanics. Below is the general form, which will be useful as it is tailored to the chemical dosage design. [(AguaClara Textbook)](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-review-review-fluid-mechanics-4)

$$\frac{p_{1}}{\rho g} + z_{1} + \alpha_{1} \frac{\bar v_{1}^2}{2g} + h_P = \frac{p_{2}}{\rho g} + z_{2} + {\alpha_{2}} \frac{\bar v_{2}^2}{2g} + h_T + h_L$$

Where:

p = pressure

ρ = density 

z = height 

v = velocity 

hp, ht = viscous frictional loss due to pumps, turbines

hl = head loss


Head loss includes major losses from friction and minor losses from flow expansions, elbows. Major and minor loss equations are discussed below. 

 $$h_L = h_{\rm{f}} + h_e$$

Minor losses may be summarized as the equation below. Minor losses occur at flow expansions, elbows, and valves. K will vary depending on the kind of minor loss. 

$$h_e = \sum K \frac{\bar v_{out}^2}{2g} =\left( \frac{A_{out}}{A_{in}} -1 \right)^2 \frac{\bar v_{out}^2}{2g}$$

The **Darcy-Weisbach equation** solves for major losses in the head loss equation. Major losses are the frictional losses that occur in pipes. For this equation, it will be important to know the length and diameter of the pipes in our delivery system. Equation found in the [AguaClara Textbook](https://aguaclara.github.io/Textbook/Review/Review_Fluid_Mechanics.html#equation-darcy-weisbach). Additionally, there is an accompanying python function in the AguaClara python package to calculate major losses.

$$h_{\rm{f}} \, = \, {\rm{f}} \frac{L}{D} \frac{\bar v^2}{2g}$$

$$h_{\rm{f}} \, = \,{\rm{f}} \frac{8}{g \pi^2} \frac{LQ^2}{D^5}$$

Such that: 

hf = major loss 

f = Darcy friction factor
 
L = pipe length

Q = pipe flow rate
 
D = pipe diameter

AC Python Function = pc.headloss_fric(FlowRate, Diam, Length, Nu, PipeRough)


Other types of energy that may be of use to our chemical dosage system will be kinetic and potential energy, listed below. These are more general, but may be important to consider, especially with respect to the site's geography and elevation.   

$$ E_{potential} = mgh \,\,\,\,\, E_{kinetic} = mv^2/2$$

Ultimately, whatever power is neeeded to run the chemical dosage system will be supplied by pumps. Knowing the amount of power required will help choose the appropriate pump for the design. Below is the equation for power. E represents energy and t represents time. 

$$ P = \frac{E}{t} $$

Lastly, mass conservation will important to consider during our calculations. This will help us better understand how much storage will be required when considering the required dosage. 

$$Q_{in}C_{in} = Q_{out}C_{out}$$





# **5. Shared Assumptions**

The Chemical team will be utilizing the [Fairmont.py](https://github.com/monroews/4590FSP/blob/master/Fairmont.py) file in conjunction with the other design teams in order to share assumptions from the Fairmont Sedimentation Tank Scope of Work document. It will also create dependency between the chemical design below and the designs of other teams.  

# **6. Major Decisions**

## **6.1. Ferric Chloride Coagulant**

The Fairmont Sedimentation Plant Scope of Work Document notes that two types of products will be used: a coagulant (specifically, ferric chloride) and emulsion polymer. Due to communication constraints with engineers at the Los Angeles Department of Water and Power, we are unlikely to fully understand the rationale behind the selection of ferric chloride as the coagulant. However, considering that it was already specified, we have decided to move forward with ferric chloride as the coagulant of choice.

The Scope of Work includes only the average dose of ferric chloride (13.14 mg/L) with no mention of the maximum or minimum. We have conservatively assumed that the maximum dose of ferric chloride will be 30 mg/L. This is slightly more than twice the average dose.

## **6.2. Emulsion Polymer**

On the other hand, the type of emulsion polymer was not specified in the Scope of Work document. There are many types and blends of polymers that are available in the water treatment market, many of which are proprietary and we do not have information about their chemical structures. 

Due to limitations of this design project, we will not be able to confidently select an emulsion polymer without bench testing capabilities. However, we do know that the polymer is going to be cationic, and there are multiple vendors available. We will research a few options from different vendors and select a representative emulsion polymer to provide an estimate for density, and viscosity. More information about the representative polymer will be provided in the design section. 

## **6.3. Other Chemicals**

It was noted by other teams that a high coagulant dosage could significantly alter the pH of the water. Since the Scope of Work did not specify the need for other chemical products, we made the assumption that the acid neutralizing capacity of raw water entering the Fairmont Sedimentation Plant is adequate, and that additional chemicals beyond the coagulant and emulsion polymer are not required.

## **6.4. Chemical Storage**

As both the Cottonwood Treatment Plant and Los Angeles Aqueduct Filtration Plant use ferric chloride, we initially thought of estimating tank volume using measurement tools from Google Earth. However, this proved to be quite a challenge and perhaps not the most accurate approach since the other plants may have different treatment goals.

Instead, we decided to work backwards from the annual masses of ferric chloride and emulsion polymer used specified by the Scope of Work to calculate the required tank volumes, assuming that they will be stored at the highest concentration commercially available.


## **6.5. Pump Selection**
The type of pumps will be chosen based on a broad range of flows outlined below. The [Pennsylvania Department of Environmental Protection's water treatment training module](http://files.dep.state.pa.us/Water/BSDW/OperatorCertification/TrainingModules/ww07_chem_feed_wb.pdf) outlines popular chemical dosing pumps. It recommends peristaltic (tube) pumps for feed rates less than 0.1 gallon per hour (gph), Solenoid metering pumps for feed rates between 0.1 and 10 gph, and positive displacement pumps for feed rates less than 600 gallons per hour. The maximum feed rate will help decide what pump will be appropriate.

For our design, we have selected a positive displacement diaphragm pump for the coagulant feed and a solenoid metering pump for the polymer feed based on the differences in feed rates. Both pump types use a flexible membrane and valves to push water through the pump chamber. The membrane stretches outward to let water into the chamber and stretches inward to displace and push the water outward. While the positive displacement diaphragm pump uses a piston, plunger or diaphragm to achieve the oscillating motion. The solenoid metering pump uses an electromagnet. Shown below is a figure provided by [Diener Precision Pumps](https://www.dienerprecisionpumps.com/en/positive-displacement-pumps.html) of the pumping mechanism.

The size of pumps necessary in each step of the treatment train will be determined in the design section. 


<p align="center"> 
<img src="https://www.dienerprecisionpumps.com/files/content%20bilder/info/190726/190726-diaphragm-pump.jpg">
</p>


## **6.6 Accessory & Safety Equipment** 

The Pennsylvania Department of Environmental Protection also recommends the accessory equipment for liquid feed systems. These additions will likely be inexpensive and will not contribute majorly to the plan view area of the design. For safety, we will also provide equipment to meet OSHA requirements. Note that we made several exclusions from the Pennsylvania Department of Environmental Protection's recommended list.

Ferric chloride has a pH of about 2 and is thus very corrosive, we made sure that relevant tubing and accessories are not brass. As such, we also decided not to use a pressure relief valve as maintenance would be an issue.

We also decided not to use anti-siphon valves as we have enough redundancy and back-up storage that tanks will not be empty. Back-pressure valves would not be needed based on the hydraulics of our system.


### 6.6.1. Pump Calibration Chamber
A pump calibration chamber is necessary for each pump to check the flow rate of the metering pumps used to deliver the coagulant and cationic polymer to the treatment plant. In order to maintain system accuracy, it is important to perform periodic checks of the flow rates and recalibrate the pumps as necessary. A simple calibration column can be used for this purpose, and the amount of time it takes for the solution to fill up from the zero mark on the column to the fill mark can be divided from the volume of the column to obtain a flow rate through that pump. One thing to note though is that FeCl3 solution may clog the column over time making the calibration imperfect so the calibration columns will have to be periodically replaced or cleaned.

Shown below is a calibration column from [McMaster Carr](https://www.mcmaster.com/4446K78-4446K78) 

<p align="center"> 
<img src="https://www.mcmaster.com/mvB/Contents/gfx/ImageCache/444/4446k71-@1x_636918023428203663.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/444/4446k71-@2x_636918023428203663.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/444/4446k71-@3x_636918023428203663.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/444/4446k71-@4x_636918023428203663.png?ver=ImageNotFound">
</p>

It is recommended that the selected calibration column for each pump take at least 30 seconds to fill up even in maximum flow rate conditions so that any localized irregularities in flow can be balanced out, and a more accurate measurement can be obtained.

At least one ball valve at the bottom and a return line at the top of the calibration column will also be necessary to complete the setup which will be located adjacent to the chemical storage tanks. 

### 6.6.2. Polymer Blending Unit
A polymer blending unit will be necessary to activate and mix the cationic emulsion polymer before distribution to the treatment train.

### 6.6.3. Spill Containment Zone 
Since FeCl3 is a corrosive chemical, steps will have to be taken to ensure the containment of any potential spills, according to OSHA regulations. The necessary spill containment zone must consist of grating, leak detection probes and chemical resistant coatings on the walls and floor. 

The proposed tank farm will be sunken into the ground to contain any spillage. The area will include grating as well as a drain to deal with excess chemicals. It is recommended that [chemical-resistant sorbent socks](https://www.mcmaster.com/spill-control/sorbents-for-harsh-chemicals/) be kept on hand to absorb any spills. These socks are very flexible and can cover tight corners well.

### 6.6.4. Eye Wash Stations
In the event that an operator accidentally gets a corrosive chemical like FeCl3 in their eyes, an eye wash station should be installed at a handy location in order to prevent permanent eye damage. One such eye wash station with the necessary piping found on [McMaster Carr](https://www.mcmaster.com/eye-wash-stations/ansi-compliant-floor-mount-eye-wash-stations-with-pipe-connection/) is shown below.
<p align="center"> 
<img src="https://www.mcmaster.com/mvB/Contents/gfx/ImageCache/556/5568t13-@1x_637074372322425986.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/556/5568t13-@2x_637074372322425986.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/556/5568t13-@3x_637074372322425986.png?ver=ImageNotFound,/mvB/Contents/gfx/ImageCache/556/5568t13-@4x_637074372322425986.png?ver=ImageNotFoundm">
</p>

### 6.6.5. Safety Showers
Safety showers should be provided on-site to deal with larger splashes of corrosive chemicals on the operator. The one shown below combines an eyewash station and safety shower and can also be found on [McMaster Carr](https://www.mcmaster.com/5557T7-5557T7)
<p align="center"> 
<img src="https://www.mcmaster.com/mvB/Contents/gfx/ImageCache/555/55505t2c1-i01bl1571005432@1x_637065843306759702.png?ver=ImageNotFound">
</p>



# **7. Design**

As the open tank farm will store ferric chloride coagulant for the rapid mix unit process as well as emulsion polymer for residuals handling, we suggest swapping the initial proposed open tank farm and truck staging areas proposed in the Fairmont Sedimentation Plant Scope of Work document. The truck staging area may remain adjacent to the residuals handling area, while the chemicals will not have to be pumped as great of a distance to the relevant unit processes.

![plant layout](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/plant_layout.png)



### **7.1. Maximum coagulant dosage in plant**

The first step in the design process is to determine the total maximum coagulant (FeCl3) dosage expected in our system. With the information at hand, there were two methods to calculate this number. The first way was to use the annual mass estimate of ferric chloride provided by the scope of work. The second method was to use the average dosage of ferric chloride being used at the Cottonwood Treatment Plant. 

#### **Method 1: Scope of Work assumption**

We used the annual mass of FeCl3 estimated in the Scope of Work to reverse engineer the average concentration of coagulant in the rapid mix treatment train. With the concentration of the stock solution, the feed rate of coagulant can be calculated by conservation of mass. We assumed that the avaerage concentration of the coagulant would not exceed twice the average concentration estimated. The necessary unit conversions have been left out of these equations as those can by easily calculated in Python using the pint library.

$$Q_{max,FeCl3} = \frac{Q_{des}C_{max,FeCl3}}{C_{stock,FeCl3}}$$
where

Q<sub>des</sub> = maximum design flow rate of the plant = 720 cfs

C<sub>max,FeCl3</sub> = maximum coagulant dose used in the plant = 30 mg/L 

C<sub>stock,FeCl3</sub> = concentration of 40% FeCl3 stock solution in tanks = 400 g/L

Normally, the lowest concentration of coagulant would be utilized for the coagulant design; however, 40% active strength of ferric chloride seemed widely available when looking for vendors. 

The total maximum coagulant dosage based on the Scope of Work Estimate was calculated to be 1454 gph (1529 mm/sec). 

#### **Method 2: Cottonwood Plant assumption**

Method 2 follows the same assumption employed in Method 1. This method uses the the average dosing rate at the Cottonwood plant from [resources.md](https://github.com/MWSteams/4590FSP/blob/master/resources.md). It was assumed the maximum concentration of the coagulant would not exceed twice the average concentration at the Cottonwood plant. 

The total maximum coagulant dosage based on the data from the Cottonwood Treatment Plant was calculated to be 407.5 gph (428.5 mm/s). 


In the end, we chose to use the estimate provided by the Scope of Work, 1454 gph (1529 mm/sec). The estimate is more conservative than the Cottonwood Plant estimate. It is also likely that this number will be closer to the numbers used by the Los Angeles Department of Water and Power engineers. 

### **7.2. Storage volume of coagulant for 30 days**

The next step is to determine the storage volume for 30 days worth of coagulant needed to run the plant. We assume that the plant is running at average conditions for this duration. The method to calculate the volume of coagulant needed over 30 days is to use the annual mass estimate of coagulant given in the Scope of Work.

$$V_{t,FeCl3} = \frac{M_{ann, FeCl3}}{C_{stock, FeCl3}} $$
where

M<sub>ann,FeCl3</sub> = annual estimate of mass of FeCl3 needed = 4142 tons/yr (from scope of work)

C<sub>stock,FeCl3</sub> = concentration of 40% FeCl3 stock solution in tanks = 400 g/L

The 30-day storage volume required based on the SOW annual estimate is 61,000 gallons (230.9 kiloliter ). 

[/Monroe/]: # (Include a discussion after your calculations explaining the significance of the results.)
  
To get a picture of how many tanks at the LAAFP would be required to store this volume, Google Earth was utilized to observe the existing LAAFP tank farm. Three types of tanks were identified. Below are calculations how many of each type of tank would be required to store the calculated volume. These estimates will help guide us when deciding on the number of tanks to choose.

Tank type 1 is the larger white tank on the left, which has an estimated storage capacity of 56.02 kilogallon (0.2121 megaliter). Tank type 2 is the smaller white tank on the left, which has an estimated storage capacity of 38.9 kilogallon (0.1473 megaliter).
![height of tank types 1, 2](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/LAAFP_tank_type_1_2_height.png)

![diameters of tank types 1, 2](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/LAAFP_tank_type_1_2_diam.png)

Tank type 3 is the large blue-green tank below, which has estimated storage capacity of 82.16 kilogallon (0.311 megaliter).
![height and diameter of tank type 3](https://raw.githubusercontent.com/MWSteams/4590FSP/master/Images/Chemicals/LAAFP_tank_type_3.png)



Considering that there are 2-3 tanks for each tank type in the Los Angeles Aqueduct Filtration Plant, the above estimates of having 3-6 tanks seem relatively reasonable. This suggests that we are moving in the right direction.

As mentioned previously, we would like a minimum 3 coagulant storage tanks for redundancy – one for use, one for back-up, and one for additional back-up during maintenance. However, if we keep the number of coagulant storage tanks to the minimum of 3, each tank would have to be very large. To provide more flexibility with storage tank options by having each coagulant storage tank hold a smaller volume, we have decided to move forward with 6 storage tanks for the coagulant. We also liked the number 6 as it is easily divisible since there will be up to 3 rapid mix units operating at once, and at least 2 feeds per treatment train. Additionally, as the ferric chloride coagulant is corrosive, we have decided to move forward with fiber glass tanks.

![coagulant tank](http://www.diamondfiberglass.com/wp-content/uploads/2019/06/107_0783.jpg)



### **7.3. Annual cost of coagulant**


According the [Brenntag Connect](https://us.brenntagconnect.com/ccrz__ProductDetails?viewState=DetailView&cartID=&portalUser=&store=&cclcl=en_US&sku=273144), our coagulant vendor of choice, showed that the cost of 40% FeCl3 solution is around $0.2243/lbs for bulk orders. This value can be used to calculate the annual cost of coagulant using the formula

$$Annual\: cost\: of\:FeCl3 = (unit\: cost\:of\:FeCl3)*V_{30,FeCl3}* \rho_{FeCl3}$$

where 

V<sub>30,FeCl3</sub> = Volume needed to store 30 days worth of 40% FeCl3 solution (calculated above)

$\rho$<sub>FeCl3</sub> = density of 40% FeCl3 solution = 1.356 g/mL ( [BrennTag Connect](https://us.brenntagconnect.com/ccrz__ProductDetails?viewState=DetailView&cartID=&portalUser=&store=&cclcl=en_US&sku=273144))

The annual cost of ferric chloride solution from the selected vendor is 1.858 million dollars. This estimate is roughly .8848 the cost that was estimated in the Scope of Work. The estimate we calculated is similar to the one provided in the Scope of Work. This makes sense since it is likely that the Scope of Work would be conservative in their budget estimates. 

### **7.4. Storage Volume & Max. Feed Rate of Emulsion Polymer**

While it may be intuitive that we do our emulsion polymer calculations similar to the coagulant calculations, we will not be employing Method 2 for these calculations. This is because the Cottonwood Treatment Plant uses their emulsion polymer for their rapid mix and coagulation processes. They do not have a separate sludge removal train. Instead, their sludge settles at the bottom of the reservoir. Since they use their emulsion polymer for different purposes, it is difficult to assume that their data would be helpful for our purposes. Additionally, it is more consistent to use the Scope of Work estimate since this is the method selected for coagulant calculations. 

The Tramfloc 300-399 Series Cationic Emulsion Polymer was selected for this design as a representative polymer. The polymer's safety data sheet may be found [here](http://tramfloc.com/material-safety-data-sheets/). This polymer was selected because it had the lowest active strength of the company's available polymers, meaning it provides the most conservative design. 

The volume needed to store polymer solution for 30 days is provided below. It is based on the definitions of volume, solution concentration, and mass. 

$$V_{month,polymer} = \frac{M_{month, polymer}}{C_{stock, polymer}} $$

Based on the Scope of Work annual mass estimate, the volume of emulsion polymer needed is 3263 gallons (12.35 kiloliter). For redundancy purposes, 2 plastic tanks of 2000 gallons were chosen to store the polymer.

The flow of the polymer is slightly more difficult to calculate since the polymer needs to be "activated". Activating the polymer includes diluting and blending the polymer. If this is not done, the polymer will create goopy "fish-eyes" which will make the polymer less effective. After the polymer leaves the storage tank it enters a blending unit where water will be added to dilute the polymer. The maximum flow of polymer out of the stock tanks was calculated to be 8.933 gph (9.393 mL/s) based on the below equation similarly used for the coagulant solution: 

$$Q_{max,polymer} = \frac{Q_{des}C_{max,polymer}}{C_{stock,polymer}}$$

where

M<sub>ann, p</sub> = annual estimate of mass of polymer needed = 165 tons/yr (from scope of work)

C<sub>stock,polymer</sub> = concentration of 35% FeCl3 stock solution in tanks =  5 g/L 

The blending unit has a maximum dilution water intake of 1200 gallons per hour. Therefore, the maximum flow rate out of the polymer feed pipes will be 1209 gph (1.271 L/s). The dilution of the emulsion polymer ultimately does not impact the overall storage design; however, it will greatly impact the nominal pipe diameter and polymer pump selection which will be detailed below. 

### **7.5. Pipe Design**

The design below calculates the optimal nominal diameter for a schedule 40 pipe. First, the function ac.diam_pipe was used to find the optimal inner diameter. This function uses the acceptable head loss and required flow to calculate the optimal inner diameter. This is then used to find the corresponding schedule 40 inner diameter and nominal diameter. The inputs the function considered were: the pipe flow, the acceptable head loss, the pipe length, the coagulant viscosity, the roughness of the pipe, and the minor losses.

The design will include 2 feed lines per rapid mix unit for redundancy purposes; however each feed line will be required to handle the full flow required per treatment train. It is also assumed that 10 m of head loss is acceptable to start. Lastly, the pipe design provides the actual head loss given the nominal diameter of the selected schedule 40 pipe. 

The coagulant stock pipes and feed pipes were calculated to have a nominal diameter of 2 inches. The water pipe for polymer dilution was calculated to have a nominal diameter of 1.5 inches. The polymer feed pipe was calculated to have a nominal diameter of 4 inches. Since the polymer dosing system has a difference in pipe diameters, fittings will be needed to connect the pipes to the diaphragm pump.

### **7.6 Pump Design**

Each treatment train will have two pumps associated with it for redundancy purposes. Each pump will be required to handle the total feed flow for the treatment train. There are 4 rapid mix units, so 8 pumps will be required for the coagulant feed system. There is one residuals treatment train, so 2 pumps will be located at each polymer feed pipe. Additionally, two pumps will be used to pump water into the polymer blending unit. The number of pumps for our design totals to 14 pumps. 

The maximum flow rate for the polymer pump was calculated to be 484.7 gph. The maximum flow rate for the water pump is 1200 gph, according to the polymer blending unit vendor. The maximum flow rate for the polymer pumps is 1204.5 gph, assuming the pump is pumping both the maximum amount of polymer and the maximum amount of water for dilution. All of these values are above the 10 gph threshold previously discussed for solenoid metering pumps. The magnitude of these pipes made it clear that positive displacement pumps will likely be needed. 

The positive displacement diaphragm pump that we have selected for the coagulant is the  [PSG All-Flo AO50](https://www.psgdover.com/all-flo/markets/chemical?matchtype=p&sncid=13&utm_source=google&utm_medium=cpc&utm_campaign=markets&adgroup=chemical&gclid=CjwKCAjw5Ij2BRBdEiwA0Frc9YyZZSjxBySLTWs4zk4dQslXvDqiDj5hAKQQbpxY0Y56FUkcucy4dhoCozwQAvD_BwE). It has a maximum capacity of 1020 gph. The positive displacement diaphragm pump that we have selected the polymer liquid feed system is the [PSG All-Flo A100](https://www.psgdover.com/all-flo/markets/chemical?matchtype=p&sncid=13&utm_source=google&utm_medium=cpc&utm_campaign=markets&adgroup=chemical&gclid=CjwKCAjw5Ij2BRBdEiwA0Frc9YyZZSjxBySLTWs4zk4dQslXvDqiDj5hAKQQbpxY0Y56FUkcucy4dhoCozwQAvD_BwE). This pump has a maximum capacity of 2460 gph. Both of these pumps are well-equipped to handle the maximum capacity of flow needed. 

### **7.7 Calibration Chamber**

We will need two types of pump calibration chambers. The first type is for the coagulant feed, the second type is for the polymer feed. The calibration chambers will be sized such that it takes at least 30 seconds for a reading at the maximum chemical flow rate.

$$t_{fill} = 30 s$$

The minimum volume of the calibration chamber for the coagulant feed pumps is determined by the following:

$$V_{min,FeCl3} = {Q_{max,FeCl3} * t_{fill}} $$

Similarly, to calculate the minimum volume of the calibration chamber for the polymer feed:

$$V_{min,polymer} = {Q_{max,polymer} * t_{fill}} $$

This method results in a minimum calibration chamber volume of 15,290 mL for the coagulant feed and 140.9 mL for the polymer feed. The chosen vendor is sufficient based on these requirements. We have selected the models with 20,000 mL and 250 mL volume capacity.





# **8. Next Steps and Challenges**

1. Reach out to the engineers at Los Angeles Department of Power and Water to confirm Scope of Work assumptions in calculating annual needs of ferric chloride coagulant and emulsion polymer, which we used as the basis of our design.

2. Reach out directly to vendors and/or representatives. This was a lower priority for our team due to the time constraint. Many of the vendors had the equipment without need for customization; however, it will be important for budget purposes to know specifically how much each product will cost.

3. Conduct bench-scale testing for residuals treatment and analyze results to select specific emulsion polymer that meets requirements.

4. Conduct a full financial analysis after receiving quotes from vendors.

5. If the elevation of the chemical storage tanks create a great enough head, it may be possible to let the chemical flow by gravity. If this is the case, the diaphragm pumps will need to be able to withstand vacuum pressure on the pump’s suction side. If the pumps cannot withstand this vacuum pressure, we recommend exploring the possibility of using day storage tanks. The day storage tanks would be located closer to the rapid mix unit, and would be connected to the long-term storage tanks via pipes. At least two one-day storage tanks would be necessary for redundancy purposes.


# **Vendors and contact information file**

See this [tab delimited spreadsheet](https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt).

Add rows to the vendor spreadsheet as you find them. The sheet can easily be edited in Excel. Just make sure to save it as a tab delimited sheet. Push your changes to github so the entire class has access to the document.



## **Appendix**

In [1]:
!pip install aguaclara
!pip install molmass

     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 163kB 15.5MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.2.1-cp36-none-any.whl size=99466 sha256=0b593e6f46052bdf3e09093ecba5f99e4fc9760164cff4b927dac5ebc5de2bfd
  Stored in directory: /root/.cache/pip/wheels/7f/ff/4f/10d76e6a261d392aefbdf6ce8d3d5519e924dc334b102ca49b
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134156 sha256=9fa6b3720c75290e8199c39f8a03a0d9bae63f6f8798d0bee400e54c57234c0b
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint
     |████████████████████████████████| 71kB 3.1MB/s 


In [2]:
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import molmass as mol
!wget https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
import Fairmont as fmt

--2020-05-19 00:11:35--  https://raw.githubusercontent.com/monroews/4590FSP/master/Fairmont.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3064 (3.0K) [text/plain]
Saving to: ‘Fairmont.py’

Fairmont.py         100%[===================>]   2.99K  --.-KB/s    in 0s      

2020-05-19 00:11:35 (21.4 MB/s) - ‘Fairmont.py’ saved [3064/3064]



In [3]:
# List of vendors compiled by all teams in CEE 4590
vendors = pd.read_csv('https://raw.githubusercontent.com/monroews/4590FSP/master/vendors.txt',sep='\t')
vendors[5:17]

,item description,vendor,contact first name,contact last name,vendor website,product info,date of contact,who made the contact,notes
5,Pump calibration column,McMaster Carr,still,pending,https://www.mcmaster.com/,https://www.mcmaster.com/4446K78-4446K78/,5/15/2020,Ananya Gangadhar,"20,000 mL for FeCl3 and 250 mL for polymer"
6,Eye wash stations,McMaster Carr,still,pending,https://www.mcmaster.com/safety-showers/,https://www.mcmaster.com/5557T7-5557T7,5/15/2020,Ananya Gangadhar,Meets OSHA requirements
7,Safety showers with eye wash stations,McMaster Carr,still,pending,https://www.mcmaster.com/eye-wash-stations/ans...,https://www.mcmaster.com/5568T13-5568T13,5/15/2020,Ananya Gangadhar,Meets OSHA requirements
8,40% FeCl3 solution,Brenntag,still,pending,https://us.brenntagconnect.com/,https://us.brenntagconnect.com/ccrz__ProductDe...,5/15/2020,Ananya Gangadhar,Maximize plan view area
9,Coagulant storage tank,Precision tank,Gary,Ruff,https://www.precisiontank.com/,https://www.precisiontank.com/products/fibergl...,5/15/2020,Bridget Childs,"Fiberglass tank, 12' Diameter"
10,Coagulant storage tank,"National Storage Tank, Inc.",still,pending,http://www.diamondfiberglass.com/,http://www.diamondfiberglass.com/fiberglass-re...,5/15/2020,Bridget Childs,"12660 Gallon Above Ground Fiberglass Tank, 12’..."
11,Coagulant storage tank,Diamond Fiberglass,still,pending,http://www.diamondfiberglass.com/,http://www.diamondfiberglass.com/fiberglass-re...,5/15/2020,Bridget Childs,"Fiberglass tank, <14’ Diameter"
12,Polymer storage tank,The Tank Depot,still,pending,https://www.tank-depot.com/,https://www.tank-depot.com/productdetails.aspx...,5/15/2020,Bridget Childs,"2000 Gallon Vertical Liquid Storage Tank, 7.6'..."
13,Cationic polymer,CARUS Water,still,pending,https://www.caruscorporation.com/,https://www.caruscorporation.com/resources/con...,5/15/2020,Bridget Childs,"POLYPRO cationic polymer: POLYPRO 4935, Densit..."
14,Diaphragm pump for coagulant feed,PSG,still,pending,https://www.psgdover.com/,https://www.psgdover.com/all-flo/markets/chemi...,5/15/2020,Bridget Childs,"Diaphragm Pump: AO50, Max flow rate: 17 gpm"


In [5]:
# Relevant parameters related to ferric chloride stock solution: 
stock_strength_fecl3 = 0.4 # Active strength of ferric chloride stock solution
stock_density_fecl3 = 1.356*u.g/(u.mL) # for 40% m/v FeCl3, Brenntag Connect 
stock_c_fecl3 = 400 * u.g/u.L # conc of coagulant stock in tank
stock_mu_fecl3 = 0.12 * u.N * u.s / u.m**2 # for 40% m/v FeCl3, Brenntag Connect
stock_nu_fecl3 = stock_mu_fecl3/stock_density_fecl3
temp = 20 *u.degC

# Relevant parameters related to diluted ferric chloride feed solution:
feed_strength_fecl3 = stock_strength_fecl3 # CHANGE IF FECL3 WILL BE DILUTED
feed_density_fecl3 = stock_density_fecl3 # CHANGES WITH DILUTION. USE LOOKUP TABLE
feed_nu_fecl3 = stock_nu_fecl3 # CHANGES WITH DILUTION. USE LOOKUP TABLE
 
# Coagulant stock flow (max, total) - Method 1 (SOW)
stock_q_fecl3 = (fmt.q_max * fmt.Fe_c_max/stock_c_fecl3).to(u.mL/u.s)
print('The total maximum ferric chloride dosing flow rate from Method 1 is',stock_q_fecl3.to(u.gal/u.hour), "(",stock_q_fecl3.to(u.mL/u.s),")")

# Coagulant stock flow (max, total) - Method 2 (Hering & Kneebone 2002 (p.169))
fecl3_dose_avg_CTP = 5.7 * u.mg/u.L 
fecl3_dose_max_CTP = fecl3_dose_avg_CTP * 2 # Assume that maximum dose is double
fecl3_mass_delivery_rate_CTP = (fmt.q_max * fecl3_dose_max_CTP)/stock_strength_fecl3
fecl3_vol_delivery_rate_CTP = fecl3_mass_delivery_rate_CTP/stock_density_fecl3
print("The total maximum feed rate of ferric chloride solution from Method 2 is", fecl3_vol_delivery_rate_CTP.to(u.gal/u.hour), "(",fecl3_vol_delivery_rate_CTP.to(u.mL/u.s),")")

# Coagulant feed rate (max, per train) 
feeds_per_train = 2 #two pipes for redundancy
num_feed_pipes = feeds_per_train*fmt.n_floc_mech #total # coagulant feed pipes
feed_q_tot_fecl3 = stock_q_fecl3* stock_strength_fecl3/feed_strength_fecl3 #total feed Q of coagulant in plant
train_q_water = fmt.q_max/(fmt.n_floc_mech-1) #Q of water in each treatment train
train_q_fecl3 = feed_q_tot_fecl3 / (fmt.n_rm_mech-1) 
t_fill = 30 * u.second # minimum amt of time it should take to fill the calibration column in order to provide accurate average flow results
cal_chamber_fecl3 = train_q_fecl3 * t_fill
print("The maximum flow rate for the raw water per train is", train_q_water.to(u.gal/u.hour), "(",train_q_water.to(u.mL/u.s),")")
print("The maximum flow rate for the coagulant feed per train is", train_q_fecl3.to(u.gal/u.hour), "(",train_q_fecl3.to(u.mL/u.s),")")
print("The minimum volume of the pump calibration chamber for coagulant feed should be", cal_chamber_fecl3.to(u.mL))

# Coagulant volume for 30 days - SOW 
ann_mass_fecl3 = 4142 * u.tons #provided by SOW
month_mass_fecl3 = ann_mass_fecl3 *u.month/u.year 
month_vol_fecl3 = month_mass_fecl3 /stock_density_fecl3
print("The volume of FeCl3 needed for 30 days based on the SOW annual estimate is", month_vol_fecl3.to(u.kgal), "(",month_vol_fecl3.to(u.kL),")")

# Coagulant annual cost
SOW_ann_cost_fecl3 = 2100000 *u.USD #from SOW estimate
unit_cost_fecl3 = 0.2243 * u.USD/u.lbs # from Brenntag vendor
ann_cost_fecl3 = unit_cost_fecl3*ann_mass_fecl3
cost_ratio = ann_cost_fecl3/SOW_ann_cost_fecl3 
print('The annual cost of FeCl3 from the selected vendor is', ann_cost_fecl3.to(u.USD))
print("The cost ratio of FeCl3 based of on the vendor's pricing is", ac.round_sig_figs(cost_ratio.to(u.dimensionless).magnitude), "the cost based on the scope of work")

# Relevant paramater of polymer stock solution:
density_polymer = 1.01*u.g/(u.mL) # Tramfloc 300-399 Series Cationic Emulsion Polymer
polymer_strength = 0.35 # Active strength of polymer solution
polymer_stock = 5 *u.g/u.L 
mu_polymer = 1 * u.N * u.s / u.m**2 
nu_polymer = mu_polymer/density_polymer

# Polymer volume for 30 days & feed rate
ann_mass_polymer = 165 * u.tons # provided by SOW
q_pw = 1200 *u.gal/u.hour # max water flow rate for PolyBlend PB 1000-2 model
month_mass_polymer = ann_mass_polymer *u.month/u.year
month_vol_polymer = month_mass_polymer/density_polymer
q_polymer_stock = 2* month_vol_polymer / u.month #assuming max polymer flow will not exceed two times month_vol_polymer
q_polymer_feed = q_polymer_stock + q_pw #max Q of the polymer pump after the blednign unit
cal_chamber_polymer = q_polymer_stock * t_fill # min volume of calibration column for polymer pumps
print("The volume needed to store polymer for 30 days is", month_vol_polymer.to(u.kgal), "(",month_vol_polymer.to(u.kL),")" )
print("The maximum flow rate of the stock polymer solution is", q_polymer_stock.to(u.gal/u.hour), "(", q_polymer_stock.to(u.L/u.sec),")" )
print("The maximum flow rate of the polymer feed pipe is", q_polymer_feed.to(u.gal/u.hour), "(", q_polymer_feed.to(u.L/u.sec),")" )
print("The minimum volume of the pump calibration chamber for polymer feed is", cal_chamber_polymer.to(u.mL))

# function to find optimal sch40 ND:
def ND_sch40(Q, head_l, pipe_l, nu, e, minor_loss): 
  diam_pipe = ac.diam_pipe(Q, head_l, pipe_l, nu, e, minor_loss) #returns pipe diameter for given headloss and Q
  av_pipe_array = ac.ID_sch40(ac.ND_all_available()) #returns all ID for sch40
  ID_sch40 = ac.ceil_nearest(diam_pipe, av_pipe_array)
  index = np.where(ID_sch40 == av_pipe_array) #index of ND and ID
  ND_array = ac.ND_all_available()
  ND_sch40 = ND_array[index][0]
  return ND_sch40

def Hl_actual(Q, head_l, pipe_l, nu, e, minor_loss): #if you're willing to have this amount of hl, design is okay.
  diam_pipe = ac.diam_pipe(Q, head_l, pipe_l, nu, e, minor_loss)
  av_pipe_array = ac.ID_sch40(ac.ND_all_available()) #returns all ID for sch40
  ID_sch40 = ac.ceil_nearest(diam_pipe, av_pipe_array)
  hl_actual = ac.headloss_pipe(Q, ID_sch40, pipe_l, nu, e, minor_loss)
  return hl_actual

# Relevant pipe design parameters:
e = 0.0015 *u.mm #Frank Fluid Mechanics pg 371, assumed plastic PVC pipe
k_elbow = 0.3 #Frank Fluid Mechanics pg 391, assumed 90 deg. regular flanged 4 in. nominal diameter
k_ball_valve = 8.5 #Frank Fluid Mechanics pg 388, assumed flanged 2 inch diameter, fully open

# Coagulant Feed Pipe: optimal sch40 ND
feed_pipe_l = 1000 *u.feet #estimate from plant schematic
feed_pipe_elbow = 4 # estimates from our proposed design
feed_pipe_valve = 2 # estimates from our proposed design
feed_pipe_hl = 10 *u.m #assumed this amount of hL is acceptable
feed_pipe_minor = feed_pipe_elbow*k_elbow + feed_pipe_valve*k_ball_valve
feed_ND_sch40 = ND_sch40(train_q_fecl3, feed_pipe_hl, feed_pipe_l, feed_nu_fecl3, e, feed_pipe_minor) # calculates nominal diameter
feed_hl_actual = Hl_actual(train_q_fecl3, feed_pipe_hl, feed_pipe_l, feed_nu_fecl3, e, feed_pipe_minor) # actual headloss from nominal diameter pipe
print("The nominal diameter of the coagulant feed pipe is",feed_ND_sch40)
print("The actual headloss of the coagulant feed pipe design is", feed_hl_actual) 

# Coagulant Stock pipe: optimal sch40 ND
stock_pipe_l = 200 *u.feet #estimate from plant schematic
stock_pipe_elbow = 4 # estimates from our proposed design
stock_pipe_valve = 2 # estimates from our proposed design
stock_pipe_hl = 10 *u.m #1 m headloss to start
stock_pipe_minor =stock_pipe_elbow*k_elbow + stock_pipe_valve*k_ball_valve #minor losses
stock_ND_sch40 = ND_sch40(stock_q_fecl3, stock_pipe_hl, stock_pipe_l, stock_nu_fecl3, e, stock_pipe_minor) # calculates nominal diameter
stock_hl_actual = Hl_actual(stock_q_fecl3, stock_pipe_hl, stock_pipe_l, stock_nu_fecl3, e, stock_pipe_minor) # actual headloss from nominal diameter pipe
print("The nominal diameter of the coagulant stock pipe is",stock_ND_sch40)
print("The actual headloss of the coagulant stock pipe design is", stock_hl_actual)

# Polymer Water Dilution pipe: optimal sch40 ND
pw_pipe_l = 100 *u.feet 
pw_pipe_elbow = 4
pw_pipe_valve = 2
pw_pipe_hl = 10 *u.m #1 m headloss to start
pw_pipe_minor = pw_pipe_elbow*k_elbow + k_ball_valve * pw_pipe_valve
q_pw = 1200 *u.gal/u.hour # max water flow rate for PolyBlend PB 1000-2 model
pw_ND_sch40 = ND_sch40(q_pw, pw_pipe_hl, pw_pipe_l, ac.viscosity_kinematic_water(temp), e, pw_pipe_minor )
pw_hl_actual = Hl_actual(q_pw, pw_pipe_hl, pw_pipe_l, ac.viscosity_kinematic_water(temp), e, pw_pipe_minor)
print("The nominal diameter of the water pipe for polymer dilution is",pw_ND_sch40)

# Polymer Stock pipe: optimal sch40 ND
polymer_pipe_l = 500 *u.feet 
polymer_pipe_elbow = 4
polymer_pipe_valve = 2
polymer_pipe_hl = 10 *u.m #1 m headloss to start
polymer_pipe_minor =  polymer_pipe_elbow*k_elbow + k_ball_valve*polymer_pipe_valve
polymer_ND_sch40 = ND_sch40(q_polymer_feed, polymer_pipe_hl, polymer_pipe_l, nu_polymer, e, polymer_pipe_minor)
polymer_hl_actual = Hl_actual(q_polymer_feed, polymer_pipe_hl, polymer_pipe_l, nu_polymer, e, polymer_pipe_minor)
print("The nominal diameter of the polymer stock pipe is",polymer_ND_sch40)
print("The actual headloss of the polymer stock pipe design is", polymer_hl_actual)


# Pump power requirements and cost calculations

def power_pump(density, Q, hl, elevation_i, elevation_f, efficiency): 
  head_pump = hl + elevation_f - elevation_i
  power_pump = (head_pump * ac.GRAVITY * Q * density)/efficiency
  return power_pump

coagulant_elevation_tank = 2 *u.feet #assumed tanks are at this elevation
polymer_elevation_tank = 2 *u.feet #assumed tanks are at this elevation
water_elevation = -2 * u.feet #assumed water mains are underground
residual_elevation = 0 *u.feet #elevation of the residuals of treatment train, lower elevation, 0 is conservative design
rapid_mix_elevation = 10 *u.feet #elevation of rapid mix units, max. elevation change based on SOW elevation map 
electricity_cost = 0.12*u.USD/(u.kW*u.hour) #estimate from Mickey & Tori
pump_efficiency = 0.5 # Conservative estimate from https://www.pumpsandsystems.com/pump-efficiency-what-efficiency

feed_pump_power = power_pump(feed_density_fecl3, train_q_fecl3, feed_hl_actual, coagulant_elevation_tank, rapid_mix_elevation, pump_efficiency)
polymer_pump_power = power_pump(density_polymer, q_polymer_feed, polymer_hl_actual, polymer_elevation_tank, residual_elevation, pump_efficiency) # power needed to pump raw polymer into mixing unit
pw_pump_power = power_pump (ac.density_water(temp), q_pw, pw_hl_actual, water_elevation, polymer_elevation_tank, pump_efficiency) # power needed to pump water into the mixing unit

pump_power_total = polymer_pump_power + pw_pump_power + feed_pump_power* (fmt.n_floc_mech - 1) 
ann_cost_pump_power = pump_power_total * electricity_cost *u.year
print("The total power required to deliver the polymer and coagulant to their respective treatment trains is", pump_power_total.to(u.watt), ". Note that this does not include the blending unit")
print("The annual cost of power for the coagulant and polymer pumps is",ann_cost_pump_power.to(u.USD),". Again, note that this does not include the blending unit")

The total maximum ferric chloride dosing flow rate from Method 1 is 1454 gallon / hour ( 1529 milliliter / second )
The total maximum feed rate of ferric chloride solution from Method 2 is 407.5 gallon / hour ( 428.5 milliliter / second )
The maximum flow rate for the raw water per train is 2.77e+06 gallon / hour ( 2.913e+06 milliliter / second )
The maximum flow rate for the coagulant feed per train is 484.7 gallon / hour ( 509.7 milliliter / second )
The minimum volume of the pump calibration chamber for coagulant feed should be 1.529e+04 milliliter
The volume of FeCl3 needed for 30 days based on the SOW annual estimate is 61 kilogallon ( 230.9 kiloliter )
The annual cost of FeCl3 from the selected vendor is 1.858e+06 dollar
The cost ratio of FeCl3 based of on the vendor's pricing is 0.8848 the cost based on the scope of work
The volume needed to store polymer for 30 days is 3.263 kilogallon ( 12.35 kiloliter )
The maximum flow rate of the stock polymer solution is 8.933 gallon / hou

In [0]:
# Utilizing LAAFP go-by's to get compare volume needed and typical tank sizes (which were obtained from Google Earth)

#LAAFP tank volume for type 1 (larger white tanks on the left)
LAAFP_tank_1_height = 7.5 * u.m
LAAFP_tank_1_diam = 6 * u.m
LAAFP_tank_1_volume = np.pi * (LAAFP_tank_1_diam/2)**2 * LAAFP_tank_1_height
LAAFP_tank_1_number = np.ceil(month_vol_fecl3.to(u.L)/LAAFP_tank_1_volume.to(u.L))
print("The volume estimate for LAAFP tank type 1 is",LAAFP_tank_1_volume.to(u.kilogallon), "(",LAAFP_tank_1_volume.to(u.ML),")")
print((LAAFP_tank_1_number.to(u.dimensionless)).magnitude, "tanks are required for tank type 1 to store", month_vol_fecl3.to(u.kL), "(",month_vol_fecl3.to(u.ML),")")

#LAAFP tank volume for type 2 (smaller white tanks on the right)
LAAFP_tank_2_height = 7.5 * u.m
LAAFP_tank_2_diam = 5 * u.m
LAAFP_tank_2_volume = np.pi * (LAAFP_tank_2_diam/2)**2 * LAAFP_tank_2_height
LAAFP_tank_2_total = LAAFP_tank_2_volume * 3
LAAFP_tank_2_number = np.ceil(month_vol_fecl3.to(u.L)/LAAFP_tank_2_volume.to(u.L))
print("The volume estimate for LAAFP tank type 2 is", LAAFP_tank_2_volume.to(u.kilogallon),"(",LAAFP_tank_2_volume.to(u.ML),")")
print((LAAFP_tank_2_number.to(u.dimensionless)).magnitude,"tanks are required for tank type 2 to store", month_vol_fecl3.to(u.kL),  "(",month_vol_fecl3.to(u.ML),")")

#LAAFP tank volume for type 3 (blue-green tanks)
LAAFP_tank_3_height = 11 * u.m
LAAFP_tank_3_diam = 6 * u.m
LAAFP_tank_3_volume = np.pi * (LAAFP_tank_3_diam/2)**2 * LAAFP_tank_3_height
LAAFP_tank_3_total = LAAFP_tank_3_volume * 2
LAAFP_tank_3_number = np.ceil(month_vol_fecl3.to(u.L)/LAAFP_tank_3_volume.to(u.L))
print("The volume estimate for LAAFP tank type 3 is", LAAFP_tank_3_volume.to(u.kilogallon),"(",LAAFP_tank_3_volume.to(u.ML),")")
print((LAAFP_tank_3_number.to(u.dimensionless)).magnitude, "tanks are required for tank type 3 to store", month_vol_fecl3.to(u.kL),  "(",month_vol_fecl3.to(u.ML),")")